# Problem Description

Design and build a least recently used cache, which evicts the least recently used item. The cache should map from keys to values (allowing you to insert and retrieve a value associated
with a particular key) and be initialized with a max size. When it is full, it should evict the least recently used item. You can assume the keys are integers and the values are strings.

### LinkedListNode Class

* Purpose: Represents a node in a doubly-linked list, which helps in maintaining the order of elements in the cache.
* Attributes:
   * key: The key associated with the node.
   * value: The value associated with the key.
   * next: Reference to the next node in the linked list.
   * prev: Reference to the previous node in the linked list.

In [7]:
class LinkedListNode:
    '''
    LinkedListNode Class:
        Represents a node in the doubly-linked list.
    '''
    def __init__(self, k:int, v:str) -> None:
        '''
        Attributes:
            key: The key of the node.
            value: The value associated with the key.
            next: Reference to the next node in the linked list.
            prev: Reference to the previous node in the linked list.
        '''
        self.key = k
        self.value = v
        self.next:LinkedListNode = None
        self.prev:LinkedListNode = None

### Cache Class

* Purpose: Represents the LRU cache and manages key-value pairs with efficient access and eviction policies.
* Attributes:
   * maxCacheSize: The maximum size of the cache.
   * map: A dictionary to quickly access nodes by their keys.
   * listHead: The head of the doubly-linked list, representing the most recently used item.
   * listTail: The tail of the doubly-linked list, representing the least recently used item.
* Methods:
   * GetValue(key): Retrieves the value for a given key and marks it as most recently used.
   * RemoveFromLinkeList(node): Removes a node from the doubly-linked list.
   * InsertAtFrontOfLinkedList(node): Inserts a node at the front of the doubly-linked list.
   * RemoveKey(key): Removes a key-value pair from the cache.
   * SetKeyValue(key, value): Inserts or updates a key-value pair in the cache, ensuring the cache's capacity is respected.

In [8]:
class Cache:
    '''
    Cache Class:
    Represents the LRU cache.
    Overview
    An LRU cache evicts the least recently used item when it reaches its maximum capacity.
    The cache should support the following operations:
        GetValue(key): Retrieve the value associated with key and mark it as most recently used.
        SetKeyValue(key, value): Insert or update the key with value and mark it as most recently used.
        Internally, it uses a combination of a hash table for O(1) lookups and a doubly-linked list to maintain the order of use.
    This implementation ensures that all operations (insertion, deletion, access) run in O(1) time complexity, making it highly efficient.
    '''
    def __init__(self, maxSize:int) -> None:
        self.maxCacheSize:int = maxSize
        self.map:dict[int, LinkedListNode] = {}
        self.listHead:LinkedListNode = None
        self.listTail:LinkedListNode = None

    def GetValue(self, key:int) -> str:
        '''
        Retrieves the value for a key and marks it as most recently used.
        Logic:
           Look up the node in the map.
           If found, move it to the front of the linked list to mark it as most recently used.
           Return the value.
        '''
        if key not in self.map:
            return None
        item:LinkedListNode = self.map[key]
        if (item == None):
            return None
        # Move to front of list to mark as most recently used
        if (item != self.listHead):
            self.RemoveFromLinkeList(item)
            self.InsertAtFrontOfLinkedList(item)
        return item.value

    def RemoveFromLinkeList(self, node:LinkedListNode) -> None:
        '''
        Removes a node from the linked list.
        Logic:
            Update the pointers of the neighboring nodes to remove the node from the linked list.
        '''
        if (node == None):
            return
        if (node.prev != None):
            node.prev.next = node.next
        if (node.next != None):
            node.next.prev = node.prev
        if (node == self.listTail):
            self.listTail = node.prev
        if (node == self.listHead):
            self.listHead = node.next


    def InsertAtFrontOfLinkedList(self, node:LinkedListNode) -> None:
        '''
        Inserts a node at the front of the linked list.
        Logic:
            Update the pointers to insert the node at the front of the linked list.
        '''
        if (self.listHead == None):
            self.listHead = node
            self.listTail = node
        else:
            self.listHead.prev = node
            node.next = self.listHead
            self.listHead = node
            self.listHead.prev = None

    def RemoveKey(self, key:int) -> bool:
        '''
        Removes a key-value pair from the cache.
        Logic:
            Remove the node from both the map and the linked list.
        '''
        if key in self.map:
            node:LinkedListNode = self.map[key]
            self.RemoveFromLinkeList(node)
            self.map.pop(key)
        return True

    def SetKeyValue(self, key:int, value:str) -> None:
        '''
        SetKeyValue Method:
        Inserts or updates a key-value pair in the cache.
        Logic:
            Remove the old value if the key already exists.
            If the cache is full, evict the least recently used item (i.e., the tail of the linked list).
            Insert the new key-value pair and update the map and linked list.
        '''
        # Remove if already there
        self.RemoveKey(key)
        # If full, remove least recently used item from cache.
        if (len(self.map) >= self.maxCacheSize and self.listTail != None):
            self.RemoveKey(self.listTail.key)
        # Insert new node
        node:LinkedListNode = LinkedListNode(key, value)
        self.map[key] = node

### Logic and Flow

1. Initialization:
   * The Cache class is initialized with a specified maximum size. It uses a dictionary (map) for O(1) key lookups and a doubly-linked list to maintain the order of items.
2. Retrieving Values (GetValue):
   * When a value is retrieved, the corresponding node is moved to the front of the linked list to mark it as the most recently used.
   * If the key is not found in the cache, None is returned.
3. Inserting Values (SetKeyValue):
   * Before inserting a new key-value pair, the method checks if the key already exists and removes the old value if necessary.
   * If the cache is full, the least recently used item (the tail of the linked list) is evicted.
   * The new key-value pair is inserted, and the node is added to the front of the linked list.
4. Maintaining Order:
   * The linked list ensures that the most recently used items are at the front (listHead), and the least recently used items are at the back (listTail).

## Example Usage

Here's an example of how to use this LRU cache:

In [9]:
# Initialize the LRU cache with a max size of 3
cache = Cache(3)

# Set key-value pairs
cache.SetKeyValue(1, "one")
cache.SetKeyValue(2, "two")
cache.SetKeyValue(3, "three")

# Retrieve values
print(cache.GetValue(1))  # Output: one
print(cache.GetValue(2))  # Output: two

# Set more values and evict least recently used item
cache.SetKeyValue(4, "four")
print(cache.GetValue(3))  # Output: None (as 3 was evicted)

# The cache now contains (in most-recent to least-recent order): 4, 2, 1

one
two
three


# Literature

The contents base on the following literature:

* Gayle Laakmann McDowell, *Cracking the Coding Interview*, [Link](https://www.crackingthecodinginterview.com/).

**Copyright**

The notebooks are provided as [Open Educational Resources](https://en.wikipedia.org/wiki/Open_educational_resources). Feel free to use the notebooks for your own purposes. The text is licensed under [Creative Commons Attribution 4.0](https://creativecommons.org/licenses/by/4.0/), the code of the IPython examples under the [MIT license](https://opensource.org/licenses/MIT).